In [5]:
# imports
import requests
from pystac_client import Client
import folium

In [6]:
STAC_API_URL = "https://2311pup4hj.execute-api.us-west-2.amazonaws.com"
RASTER_API_URL = "https://ducbzwnzhb.execute-api.us-west-2.amazonaws.com"

In [7]:
catalog = Client.open(STAC_API_URL)

In [8]:
collection_id = "social-vulnerability-index-overall"
search = catalog.search(collections=[collection_id])
items = list(search.items())
items

[<Item id=svi_2018_tract_overall_wgs84_cog>,
 <Item id=svi_2016_tract_overall_wgs84_cog>,
 <Item id=svi_2014_tract_overall_wgs84_cog>,
 <Item id=svi_2010_tract_overall_wgs84_cog>,
 <Item id=svi_2000_tract_overall_wgs84_cog>]

In [9]:
s3_uri = items[0].assets["cog_default"].href

In [10]:
stats = requests.get(
    f"{RASTER_API_URL}/cog/statistics",
    params={"url": s3_uri}
).json()
stats

{'b1': {'min': 0.0,
  'max': 0.9998000264167786,
  'mean': 0.5843708210592843,
  'count': 92166.0,
  'sum': 53859.12109375,
  'std': 0.27337772204503596,
  'median': 0.5983999967575073,
  'majority': 0.9014999866485596,
  'minority': 9.999999747378752e-05,
  'unique': 8143.0,
  'histogram': [[3361.0,
    6051.0,
    8673.0,
    8752.0,
    9476.0,
    9934.0,
    10225.0,
    8652.0,
    7939.0,
    19103.0],
   [0.0,
    0.09998000413179398,
    0.19996000826358795,
    0.2999400198459625,
    0.3999200165271759,
    0.4999000132083893,
    0.599880039691925,
    0.6998600363731384,
    0.7998400330543518,
    0.8998200297355652,
    0.9998000264167786]],
  'valid_percent': 18.63,
  'masked_pixels': 402426.0,
  'valid_pixels': 92166.0,
  'percentile_2': 0.07612999975681305,
  'percentile_98': 0.9794999957084656}}

In [11]:
rescale = f"{stats['b1']['min']},{stats['b1']['max']}"

In [12]:
tiles = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={collection_id}&item={items[0].id}&assets=cog_default&colormap_name=rdbu_r&rescale={rescale}"
).json()

In [13]:
tiles

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://ducbzwnzhb.execute-api.us-west-2.amazonaws.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=social-vulnerability-index-overall&item=svi_2018_tract_overall_wgs84_cog&assets=cog_default&colormap_name=rdbu_r&rescale=0.0%2C0.9998000264167786'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-178.23333334, 18.908332897999998, -66.958333785, 71.383332688],
 'center': [-122.59583356249999, 45.145832793, 0]}

In [14]:
m = folium.Map(
    zoom_start=6,
    scroll_wheel_zoom=True, 
    tiles=tiles["tiles"][0], 
    attr="VEDA", 
    minzoom=0, 
    maxzoom=18,
)

In [15]:
m